In [0]:
import re
import string
from collections import Counter
import squarify
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer

from bs4 import BeautifulSoup
import html as ihtml

import requests
import sqlite3

In [0]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [0]:
def extract_company(soup):
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-linl-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)

In [0]:
def extract_location(soup):
    locations = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            locations.append(div.find(name="span", attrs={"class":"location"}).text)
        except:
            locations.append("None")
    return(locations)

In [0]:
def extract_salary(soup):
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find(name="span", attrs={"class":"salaryText"}).text.replace("\n",""))
        except:
            salaries.append("None")
    return(salaries)

In [0]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [0]:
def extract_url(soup):
    urls = []
    for div1 in soup.find_all(name="div", attrs={"class":"row"}):
        for div2 in div1.find_all(name="div", attrs={"class":"title"}):
            for a in div2.find_all(name="a", href=True):
                urls.append(a['href'])
    return(urls)

def extract_desc(urls):
    descs = []
    for url in urls:
        full_url = "https://www.indeed.com" + url
        detail_page = requests.get(full_url)
        detail_soup = BeautifulSoup(detail_page.text, "html.parser")
        
        for div in detail_soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
            descs.append(div.text)
    return(descs)

In [0]:
def extract_count(soup):
    for div in soup.find_all(name="div", attrs={"id":"searchCountPages"}):
        temp_str = div.text.replace(" ", "")
        temp_count_str = re.search('of(.*)jobs', temp_str)
        count_str = re.sub('[^0-9]','', temp_count_str.group(1))
        count = int(count_str)
    return(count)

In [0]:
def extract_list(title_name, city_name, st_name):
    max_results = 100
    columns = ["city", "job_title", "company", "location", "salary", "description"]

    city_url = "https://www.indeed.com/jobs?q=" + title_name + \
               "&l=" + city_name + "%2C+" + st_name
    page = requests.get(city_url)
    soup = BeautifulSoup(page.text, "html.parser")
    max_counter = extract_count(soup)
    print(max_counter, title_name, city_name)
   
    job_title_list = []
    company_list = []
    location_list = []
    salary_list = []
    desc_list = []

    for start in range(0, max_results, 10):
        city_url = "https://www.indeed.com/jobs?q=" + title_name + \
                   "&l=" + city_name + "%2C+" + st_name + \
                   "&start=" + str(start)
        page = requests.get(city_url)
        soup = BeautifulSoup(page.text, "html.parser")

        job_title_list.extend(extract_job_title(soup))
        company_list.extend(extract_company(soup))
        location_list.extend(extract_location(soup))
        salary_list.extend(extract_salary(soup))
        add_urls = extract_url(soup)
        desc_list.extend(extract_desc(add_urls))

    return job_title_list, company_list, location_list, salary_list, desc_list, max_counter

def job_db(title, city, st):   
    j_list, c_list, l_list, s_list, d_list, m_counter = extract_list(title, city, st)     
    temp_df = pd.DataFrame(list(zip(j_list, c_list, l_list, s_list, d_list)), 
                          columns = ['job_title', 'company', 'location', 'salary', 'description'])
    temp_df['counts'] = m_counter
    city_name = city.replace('+', ' ')
    temp_df['city'] = city_name
    return temp_df

In [11]:
temp1_df = job_db('data+scientist', 'San+Jose', 'CA')
temp2_df = job_db('data+scientist', 'San+Francisco', 'CA')
temp3_df = job_db('data+scientist', 'Seattle', 'WA')
temp4_df = job_db('data+scientist', 'Washington', 'DC')
temp5_df = job_db('data+scientist', 'New+York', 'NY')
temp6_df = job_db('data+scientist', 'Baltimore', 'MD')
temp7_df = job_db('data+scientist', 'Boulder', 'CO')
temp8_df = job_db('data+scientist', 'San+Diego', 'CA')
temp9_df = job_db('data+scientist', 'Denver', 'CO')
temp10_df = job_db('data+scientist', 'Huntsville', 'AL')
temp11_df = job_db('data+scientist', 'Colorado+Springs', 'CO')
temp12_df = job_db('data+scientist', 'Houston', 'TX')
temp13_df = job_db('data+scientist', 'Trenton', 'NJ')
temp14_df = job_db('data+scientist', 'Dallas', 'TX')
temp15_df = job_db('data+scientist', 'Columbus', 'OH')
temp16_df = job_db('data+scientist', 'Austin', 'TX')
temp17_df = job_db('data+scientist', 'Philadelphia', 'PA')
temp18_df = job_db('data+scientist', 'Durham', 'NC')
temp19_df = job_db('data+scientist', 'Raleigh', 'NC')
temp20_df = job_db('data+scientist', 'Atlanta', 'GA')

data_scientist_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                       temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                       temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                       temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)

print(data_scientist_df.shape)

1304 data+scientist San+Jose
934 data+scientist San+Francisco
1107 data+scientist Seattle
1162 data+scientist Washington
823 data+scientist New+York
163 data+scientist Baltimore
158 data+scientist Boulder
161 data+scientist San+Diego
175 data+scientist Denver
21 data+scientist Huntsville
7 data+scientist Colorado+Springs
99 data+scientist Houston
91 data+scientist Trenton
450 data+scientist Dallas
74 data+scientist Columbus
185 data+scientist Austin
216 data+scientist Philadelphia
148 data+scientist Durham
150 data+scientist Raleigh
255 data+scientist Atlanta
(3368, 7)


In [12]:
temp1_df = job_db('web+developer', 'San+Jose', 'CA')
temp2_df = job_db('web+developer', 'San+Francisco', 'CA')
temp3_df = job_db('web+developer', 'Seattle', 'WA')
temp4_df = job_db('web+developer', 'Washington', 'DC')
temp5_df = job_db('web+developer', 'New+York', 'NY')
temp6_df = job_db('web+developer', 'Baltimore', 'MD')
temp7_df = job_db('web+developer', 'Boulder', 'CO')
temp8_df = job_db('web+developer', 'San+Diego', 'CA')
temp9_df = job_db('web+developer', 'Denver', 'CO')
temp10_df = job_db('web+developer', 'Huntsville', 'AL')
temp11_df = job_db('web+developer', 'Colorado+Springs', 'CO')
temp12_df = job_db('web+developer', 'Houston', 'TX')
temp13_df = job_db('web+developer', 'Trenton', 'NJ')
temp14_df = job_db('web+developer', 'Dallas', 'TX')
temp15_df = job_db('web+developer', 'Columbus', 'OH')
temp16_df = job_db('web+developer', 'Austin', 'TX')
temp17_df = job_db('web+developer', 'Philadelphia', 'PA')
temp18_df = job_db('web+developer', 'Durham', 'NC')
temp19_df = job_db('web+developer', 'Raleigh', 'NC')

web_developer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                              temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                              temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                              temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(web_developer_df.shape)

1371 web+developer San+Jose
1415 web+developer San+Francisco
1417 web+developer Seattle
3385 web+developer Washington
1821 web+developer New+York
877 web+developer Baltimore
438 web+developer Boulder
363 web+developer San+Diego
596 web+developer Denver
78 web+developer Huntsville
68 web+developer Colorado+Springs
368 web+developer Houston
223 web+developer Trenton
771 web+developer Dallas
225 web+developer Columbus
656 web+developer Austin
530 web+developer Philadelphia
384 web+developer Durham
367 web+developer Raleigh
(3623, 7)


In [13]:
temp1_df = job_db('ux+designer', 'San+Jose', 'CA')
temp2_df = job_db('ux+designer', 'San+Francisco', 'CA')
temp3_df = job_db('ux+designer', 'Seattle', 'WA')
temp4_df = job_db('ux+designer', 'Washington', 'DC')
temp5_df = job_db('ux+designer', 'New+York', 'NY')
temp6_df = job_db('ux+designer', 'Baltimore', 'MD')
temp7_df = job_db('ux+designer', 'Boulder', 'CO')
temp8_df = job_db('ux+designer', 'San+Diego', 'CA')
temp9_df = job_db('ux+designer', 'Denver', 'CO')
temp10_df = job_db('ux+designer', 'Huntsville', 'AL')
temp11_df = job_db('ux+designer', 'Colorado+Springs', 'CO')
temp12_df = job_db('ux+designer', 'Houston', 'TX')
temp13_df = job_db('ux+designer', 'Trenton', 'NJ')
temp14_df = job_db('ux+designer', 'Dallas', 'TX')
temp15_df = job_db('ux+designer', 'Columbus', 'OH')
temp16_df = job_db('ux+designer', 'Austin', 'TX')
temp17_df = job_db('ux+designer', 'Philadelphia', 'PA')
temp18_df = job_db('ux+designer', 'Durham', 'NC')
temp19_df = job_db('ux+designer', 'Raleigh', 'NC')
temp20_df = job_db('ux+designer', 'Atlanta', 'GA')

ux_designer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                            temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                            temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                            temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(ux_designer_df.shape)

633 ux+designer San+Jose
872 ux+designer San+Francisco
944 ux+designer Seattle
522 ux+designer Washington
870 ux+designer New+York
88 ux+designer Baltimore
132 ux+designer Boulder
105 ux+designer San+Diego
146 ux+designer Denver
8 ux+designer Huntsville
6 ux+designer Colorado+Springs
82 ux+designer Houston
41 ux+designer Trenton
177 ux+designer Dallas
47 ux+designer Columbus
275 ux+designer Austin
145 ux+designer Philadelphia
83 ux+designer Durham
83 ux+designer Raleigh
252 ux+designer Atlanta
(3099, 7)


In [14]:
temp1_df = job_db('ios+developer', 'San+Jose', 'CA')
temp2_df = job_db('ios+developer', 'San+Francisco', 'CA')
temp3_df = job_db('ios+developer', 'Seattle', 'WA')
temp4_df = job_db('ios+developer', 'Washington', 'DC')
temp5_df = job_db('ios+developer', 'New+York', 'NY')
temp6_df = job_db('ios+developer', 'Baltimore', 'MD')
temp7_df = job_db('ios+developer', 'Boulder', 'CO')
temp8_df = job_db('ios+developer', 'San+Diego', 'CA')
temp9_df = job_db('ios+developer', 'Denver', 'CO')
temp10_df = job_db('ios+developer', 'Huntsville', 'AL')
temp11_df = job_db('ios+developer', 'Colorado+Springs', 'CO')
temp12_df = job_db('ios+developer', 'Houston', 'TX')
temp13_df = job_db('ios+developer', 'Trenton', 'NJ')
temp14_df = job_db('ios+developer', 'Dallas', 'TX')
temp15_df = job_db('ios+developer', 'Columbus', 'OH')
temp16_df = job_db('ios+developer', 'Austin', 'TX')
temp17_df = job_db('ios+developer', 'Philadelphia', 'PA')
temp18_df = job_db('ios+developer', 'Durham', 'NC')
temp19_df = job_db('ios+developer', 'Raleigh', 'NC')
temp20_df = job_db('ios+developer', 'Atlanta', 'GA')

ios_developer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                              temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                              temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                              temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(ios_developer_df.shape)

751 ios+developer San+Jose
468 ios+developer San+Francisco
484 ios+developer Seattle
351 ios+developer Washington
377 ios+developer New+York
82 ios+developer Baltimore
106 ios+developer Boulder
83 ios+developer San+Diego
129 ios+developer Denver
15 ios+developer Huntsville
6 ios+developer Colorado+Springs
53 ios+developer Houston
24 ios+developer Trenton
153 ios+developer Dallas
28 ios+developer Columbus
174 ios+developer Austin
88 ios+developer Philadelphia
47 ios+developer Durham
46 ios+developer Raleigh
142 ios+developer Atlanta
(3185, 7)


In [0]:
data_scientist_df['job'] = 'data scientist'
web_developer_df['job'] = 'web developer'
ux_designer_df['job'] = 'ux designer'
ios_developer_df['job'] = 'ios developer'
indeed_df = pd.concat([data_scientist_df, web_developer_df, ux_designer_df, ios_developer_df], ignore_index=True)
indeed_df = indeed_df.drop_duplicates(keep=False) 
indeed_df = indeed_df.reset_index(drop=True)

In [16]:
def convert_salary(sal_str, s_flag):
    
    sal_split = re.findall(r'\d+', sal_str.replace(",", ""))
    
    if len(sal_split) == 2:
        low_salary = int(sal_split[0])
        high_salary = int(sal_split[1])
    else:
        low_salary = None
        high_salary = None
    
    if s_flag == 'l':
        salary = low_salary
    else:
        salary = high_salary
    
    return salary

indeed_df['low_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'l'), axis=1)
indeed_df['high_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'h'), axis=1)
indeed_df = indeed_df.drop(columns=['salary'])

indeed_df.head()                                      

,job_title,company,location,description,counts,city,job,low_salary,high_salary
0,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",Help build technology that saves lives!\n\nWe'...,1304,San Jose,data scientist,NaN,NaN
1,Data Scientist,Palo Verde Consulting,"Campbell, CA 95008","Job Title: Data ScientistLocation: Campbell, C...",1304,San Jose,data scientist,150000.0,210000.0
2,Data Scientist,Spry Health,"Palo Alto, CA",Spry Health’s mission is to build the world’s ...,1304,San Jose,data scientist,100000.0,135000.0
3,Data Scientist,SLEEP NUMBER,"San Jose, CA 95123 (Blossom Valley area)","Position Purpose:\nAs Data Scientist, you will...",1304,San Jose,data scientist,NaN,NaN
4,"Data Scientist in Santa Clara, CA (corp-corp c...",Advantine Technologies,"Santa Clara, CA",Job Description\n\nTitle : Data Scientist\nLoc...,1304,San Jose,data scientist,NaN,NaN


In [17]:
indeed_df.tail()

,job_title,company,location,description,counts,city,job,low_salary,high_salary
4796,Senior Software Engineer,MDT Software,"Alpharetta, GA 30005",MDT is seeking a Senior Software Engineer who ...,142,Atlanta,ios developer,NaN,NaN
4797,Software Engineer,ASSA ABLOY Opening Solutions,"Marietta, GA",HID Global powers the trusted identities of th...,142,Atlanta,ios developer,NaN,NaN
4798,Software Engineer,Assa Abloy,"Marietta, GA",HID Global powers the trusted identities of th...,142,Atlanta,ios developer,NaN,NaN
4799,Devops Engineer,MacStadium,"Atlanta, GA 30305 (Buckhead area)",Overview\nMacStadium is the leading managed Ma...,142,Atlanta,ios developer,NaN,NaN
4800,Sr. UX Designer,"Cooper Holdings, Inc.","Atlanta, GA 30339",You Are Here (YAH) is a creative agency built ...,142,Atlanta,ios developer,NaN,NaN


In [0]:
# convert to sqlite3 and csv
conn = sqlite3.connect('techsearch.sqlite3')
curs = conn.cursor()
curs.execute('drop table if exists listings')
indeed_df.to_sql('listings', con=conn)
indeed_df.to_csv('/content/techsearch.csv')